# Prerequisite

In [2]:
# Import library
import numpy as np
import pandas as pd
import cvxopt
import matplotlib.pyplot as plt
import time
import wrds

In [3]:
con = wrds.Connection()

Loading library list...
Done


In [6]:
crspd = con.raw_sql(
    """SELECT *
    FROM crsp.dsf
    """
)

In [7]:
crspd.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx
0,68391610,10000,7952,10396,3,3990,1986-01-07,2.375,2.75,-2.5625,1000.0,<NA>,<NA>,<NA>,3680.0,1.0,1.0,<NA>,<NA>,<NA>
1,68391610,10000,7952,10396,3,3990,1986-01-08,2.375,2.625,-2.5,12800.0,-0.02439,<NA>,<NA>,3680.0,1.0,1.0,<NA>,<NA>,-0.02439
2,68391610,10000,7952,10396,3,3990,1986-01-09,2.375,2.625,-2.5,1400.0,0.0,<NA>,<NA>,3680.0,1.0,1.0,<NA>,<NA>,0.0
3,68391610,10000,7952,10396,3,3990,1986-01-10,2.375,2.625,-2.5,8500.0,0.0,<NA>,<NA>,3680.0,1.0,1.0,<NA>,<NA>,0.0
4,68391610,10000,7952,10396,3,3990,1986-01-13,2.5,2.75,-2.625,5450.0,0.05,<NA>,<NA>,3680.0,1.0,1.0,<NA>,<NA>,0.05


In [8]:
crspd.describe()

,permno,permco,issuno,hexcd,hsiccd,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx
count,107663470.0,107663470.0,107663470.0,107663470.0,107484613.0,105859566.0,105822104.0,105823789.0,99468387.0,105780937.0,70531730.0,70531228.0,107663469.0,107663469.0,107663469.0,66746487.0,35300102.0,105780937.0
mean,51955.76316,22925.146477,3202613.246768,2.160755,5043.395528,36.879326,37.734397,33.053643,473429.487795,0.000716,46.083496,46.38821,47987.554917,6.051985,4.370797,48.89016,2155.607936,0.000628
std,29642.739339,16843.80763,14706297.692476,1.056718,2143.859615,1999.089428,2029.360391,2014.334338,4089680.044592,0.043019,2464.508882,2467.987396,240814.245299,109.183793,48.323433,2537.35352,15457.809003,0.043044
min,10000.0,2.0,0.0,1.0,100.0,0.0001,0.0014,-18800.0,0.0,-0.995707,0.0001,0.0007,0.0,0.0,0.0,0.0001,0.0,-0.995707
25%,20469.0,9132.0,0.0,1.0,3520.0,5.99,6.25,2.875,1618.0,-0.010909,6.62,6.83,2382.0,1.0,1.0,7.45,7.0,-0.01105
50%,53495.0,21123.0,1458.0,2.0,5047.0,14.5,15.0,12.831,15900.0,0.0,15.3125,15.6,8324.0,1.0,1.0,16.875,51.0,0.0
75%,80565.0,30516.0,16259.0,3.0,6726.0,28.62,29.29,27.5,134755.0,0.010336,30.42,30.74,29360.0,1.5,1.44375,33.14,631.0,0.01025
max,93436.0,60123.0,99995085.0,5.0,9999.0,723050.0,741971.375,724040.0,2144903939.0,39.725304,724160.0,725500.0,29206400.0,32943.925781,9899.818359,730090.8125,4897701.0,39.725304


In [11]:
crspd.to_csv('crspd.csv')

# Net Asset Value

$\mathbf{w}_{t}^{\text{cap}} = \mathbf{w}_{t-1}^{\text{cap}} \odot \left( \mathbf{p}_{t} \oslash \mathbf{p}_{t-1} \right) = \mathbf{w}_{t-1}^{\text{cap}} \odot \left( 1 + r_{t}^{\text{lin}} \right),$

where, $\mathbf{w}_{t}^{\text{cap}}$ is the capital weights at time $t$, $\mathbf{p}_{t}$ is the price vector at time $t$, $r_{t}^{\text{lin}}$ is the linear return at time $t$, and $\odot$ and $\oslash$ are element-wise multiplication and division, respectively.

$NAV_t = 1^{\top}w_t^{\text{cap}} + c_t^{\text{cap}}$